# Feature-Aware Data Augmentation

Augmentation policies to consider:
- Simple policy randomly sampling of n transforms 
- Constrained sampling policy with a blacklist of transforms to avoid 
- Feature-aware augmentation policy where transforms are picked based on their (transform, feature) behavior

Feature-Aware Data Augmentation (FADA)
1. Extract AMRs for all examples in the dataset
2. Create feature matrix by checking for the presence of different features in the text
3. Create sampling probabilities based on the feature vector
    - Select subsets of the dataset based on the presence / absence of the feature
    - Test a model already fine-tuned on the target dataset
    - Calculate disparate impact of each (feature, transform) pair
    - Normalize disparate impact into a probability vector for each sample

In [1]:
# data
import os
import pickle
import numpy as np
from datasets import load_dataset

# amrs
import amrlib
import penman

# transform
import sibyl
import torch
import inspect
import random
from functools import partial

# eval pipeline
import pandas as pd
from transformers import pipeline
from huggingface_hub import HfApi, ModelFilter
from sklearn.metrics import accuracy_score
from sibyl import acc_at_k

In [ ]:
torch.use_deterministic_algorithms(False)

In [2]:
class Transform:
    def __init__(self, transform_class, num_outputs=1, task_name="sentiment"):
        self.transform_class = transform_class
        self.num_outputs = num_outputs
        self.task_name = task_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.intakes_target = False
        self.is_batched = False
        
        # setting class attributes
        if 'to_tense' in inspect.signature(self.transform_class).parameters:
            print("initializing class with to_tense='past'") # future & random don't work
            self.transform_instance = self.transform_class(to_tense="past")
        elif 'source_lang' in inspect.signature(self.transform_class).parameters:
            print("initializing class with source_lang='es'") 
            self.transform_instance = self.transform_class(source_lang="es")
        elif 'task_name' in inspect.signature(self.transform_class).parameters:
            print(f"initializing class with task_name='{task_name}', return_metadata=True") 
            self.transform_instance = self.transform_class(task_name=self.task_name, return_metadata=True)
        # elif isinstance(self.transform_class, LostInTranslation):
        #     self.transform_instance = self.transform_class(device=0)
        else:
            self.transform_instance = self.transform_class()
        
        # setting instance attributes
        if hasattr(self.transform_instance, "max_outputs"):
            print(f"setting max_outputs={self.num_outputs}")
            self.transform_instance.max_outputs = self.num_outputs
        if hasattr(self.transform_instance, "max_paraphrases"):
            print(f"setting max_paraphrases={self.num_outputs}")
            self.transform_instance.max_paraphrases = self.num_outputs
        if hasattr(self.transform_instance, "device"):
            if self.transform_instance.device is None or self.transform_instance.device == 'cpu':
                print(f"setting device={self.device}")
                self.transform_instance.device = self.device
        
        # selecting the transformation function
        if hasattr(self.transform_class, "generate"):
            self.transform_fn = self.transform_instance.generate
        if hasattr(self.transform_class, "augment"):
            self.transform_fn = self.transform_instance.augment
        if hasattr(self.transform_class, "transform_batch"):
            self.transform_fn = self.transform_instance.transform_batch
            self.intakes_target = True
            self.is_batched = True
            
    def synced_shuffle(self, list1, list2):
        # Shuffle two lists with same order
        # Using zip() + * operator + shuffle()
        temp = list(zip(list1, list2))
        random.shuffle(temp)
        res1, res2 = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        res1, res2 = list(res1), list(res2)
        return res1, res2
            
    def apply(self, texts, labels=None):
        if self.intakes_target:
            if self.is_batched:
                new_texts, new_labels = self.transform_fn((texts, labels))
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t, new_l = self.transform_fn(t, l)
                    new_texts.append(new_t)
                    new_labels.extend([new_l] * len(new_t))
        else:
            if self.is_batched:
                new_texts = self.transform_fn((texts))
                new_texts = labels
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t = self.transform_fn(t)
                    if len(new_t) > self.num_outputs:
                        new_t = new_t[:self.num_outputs]
                    new_texts.extend(new_t)
                    new_labels.extend([l] * len(new_t))
                    
        # post processing since some transformations add/remove more new outputs than expected
        if len(new_texts) == 0:
            print("no new_texts, substituting original texts...")
            new_texts = texts
        if len(new_labels) == 0:
            print("no new_labels, substituting original labels...")
            new_labels = labels
        new_texts, new_labels = self.synced_shuffle(new_texts, new_labels)
        
        expected_len = len(texts) * self.num_outputs
        new_texts = new_texts[:expected_len]
        new_labels = new_labels[:expected_len]
        
        return new_texts, new_labels
    
def augment_data(batch, transform, keep_originals=True):
    new_texts, new_labels = [], []
    for text, label in zip(batch['text'], batch['label']):
        new_text, new_label = transform.apply([text], [label])
        new_texts.extend(new_text)
        new_labels.extend(new_label)
    if keep_originals:
        return {"text": batch['text'] + new_texts, "label": batch['label'] + new_labels}
    else:
        return {"text": new_texts, "label": new_labels}

In [3]:
# attributes =============================================================

def contains_imperative(g): return g.contains_attribute("imperative")
def contains_exlamation(g): return g.contains_attribute("expressive")
def contains_negation(g):   return g.contains_attribute("-")

# concepts ===============================================================

def contains_conjunctions(g):         return g.contains_concept(["and", "or", "contrast-01", "either", "neither"])
def contains_interrogative_clause(g): return g.contains_concept("truth-value")
def contains_question(g):             return g.contains_concept(["amr-unknown", "amr-choice"])

# roles ==================================================================

def contains_coreferences(g): return any(r for r in g.amr_text.split() if r in ['i', 'you', 'he', 'she', 'it', 'we', 'they'])
def contains_number(g):       return any(a for a in g.graph.attributes() if a.target.isnumeric())

def contains_accompanier(g):  return g.contains_role(':accompanier')
def contains_age(g):          return g.contains_role(':age')
def contains_beneficiary(g):  return g.contains_role(':beneficiary')
def contains_concession(g):   return g.contains_role(':concession')
def contains_condition(g):    return g.contains_role(':condition')
def contains_consist_of(g):   return any(r for r in g.amr_text.split() if r in [':consist-of'])
def contains_degree(g):       return g.contains_role(':degree')
def contains_destination(g):  return g.contains_role(':destination')
def contains_direction(g):    return g.contains_role(':direction')
def contains_domain(g):       return g.contains_role(':domain')
def contains_duration(g):     return g.contains_role(':duration')
def contains_example(g):      return g.contains_role(':example')
def contains_extent(g):       return g.contains_role(':extent')
def contains_frequency(g):    return g.contains_role(':frequency')
def contains_instrument(g):   return g.contains_role(':instrument')
# def contains_li(g):           return g.contains_role(':li')
def contains_location(g):     return g.contains_role(':location')
def contains_manner(g):       return g.contains_role(':manner')
def contains_medium(g):       return g.contains_role(':medium')
def contains_mod(g):          return g.contains_role(':mod')
def contains_mode(g):         return any(a for a in g.graph.attributes() if ":mode" in a.role)
def contains_name(g):         return g.contains_role(':name')
def contains_ord(g):          return g.contains_role(':ord')
def contains_part(g):         return g.contains_role(':part')
def contains_path(g):         return g.contains_role(':path')
def contains_polarity(g):     return g.contains_role(':polarity')
def contains_polite(g):       return any(r for r in g.amr_text.split() if r in [':polite'])
def contains_poss(g):         return g.contains_role(':poss')
def contains_purpose(g):      return g.contains_role(':purpose')
def contains_quant(g):        return g.contains_role(':quant')
def contains_range(g):        return g.contains_role(':range')
def contains_scale(g):        return g.contains_role(':scale')
def contains_source(g):       return g.contains_role(':source')
def contains_subevent(g):     return g.contains_role(':subevent')
def contains_time(g):         return g.contains_role(':time')
def contains_topic(g):        return g.contains_role(':topic')
def contains_unit(g):         return g.contains_role(':unit')
# def contains_value(g):        return g.contains_role(':value')
def contains_wiki(g):         return g.contains_role(':wiki')

In [103]:
class AMRGraph:
    def __init__(self, amr):
        self.graph = penman.decode(amr) if not isinstance(amr, penman.graph.Graph) else amr
        self.amr_text = penman.encode(self.graph)

    def contains_concept(self, concepts):
        """
        Concepts are nodes / instances in the AMR graph.
        """
        if not isinstance(concepts, list): concepts = [concepts]
        graph_concepts = [t.target for t in self.graph.instances()]
        return any(c for c in graph_concepts if c in concepts)

    def contains_role(self, roles):
        """
        Roles are edges in the AMR graph.
        """
        if not isinstance(roles, list): roles = [roles]
        graph_roles = [e.role for e in self.graph.edges()]
        return any(r for r in graph_roles if r in roles)

    def contains_attribute(self, attributes):
        """
        Attributes are properties of concept nodes, i.e. relationships to 
        constant values.
        """
        if not isinstance(attributes, list): attributes = [attributes]
        graph_attrs = [a.target for a in self.graph.attributes()]
        return any(a for a in graph_attrs if a in attributes)
    
class AMRGraphManager:
    def __init__(self):
        self.graphs_save_path = "amr_graphs.pkl"
        self.feature_matrix_save_path = "feature_matrix.npy"
        
        if os.path.exists(self.graphs_save_path):
            self.load_graphs()
        else:
            self.graphs = None
            self.model = None
            
        if os.path.exists(self.feature_matrix_save_path):
            self.load_feature_matrix()
        else:
            self.feature_matrix = None
        
        self.featurizers = [    
            contains_imperative,contains_exlamation,contains_negation,
            contains_conjunctions,contains_interrogative_clause,contains_question,
            contains_coreferences,contains_number,contains_accompanier,
            contains_age,contains_beneficiary,contains_concession,
            contains_condition,contains_consist_of,contains_degree,
            contains_destination,contains_direction,contains_domain,
            contains_duration,contains_example,contains_extent,
            contains_frequency,contains_instrument,contains_location,
            contains_manner,contains_medium,contains_mod,
            contains_mode,contains_name,contains_ord,
            contains_part,contains_path,contains_polarity,
            contains_polite,contains_poss,contains_purpose,
            contains_quant,contains_range,contains_scale,
            contains_source,contains_subevent,contains_time,
            contains_topic,contains_unit,contains_wiki
        ]
    
    def load_model(self, max_sent_len=128):
        self.model = amrlib.load_stog_model(max_sent_len=max_sent_len)
    
    def text_to_amr(self, texts):
        if self.model is None:
            self.load_model()
        if self.graphs is None:
            amr_penmans = self.model.parse_sents(texts, add_metadata=False, disable_progress=False)
            amr_graphs  = [AMRGraph(p) for p in amr_penmans]
            self.graphs = amr_graphs
        return self.graphs
    
    def generate_feature_matrix(self, graphs):
        if self.feature_matrix is None:
            feature_matrix = []
            for g in graphs:
                feature_vector = []
                for f in self.featurizers:
                    feature_vector.append(f(g))
                feature_matrix.append(feature_vector)
            self.feature_matrix = np.array(feature_matrix, dtype=np.bool_)
        return self.feature_matrix

    def save_graphs(self):
        with open(self.graphs_save_path, "wb") as f:
            pickle.dump(self.graphs, f)
            
    def load_graphs(self):
        with open(self.graphs_save_path, "rb") as f:
            self.graphs = pickle.load(f)

    def save_feature_matrix(self):
        np.save(self.feature_matrix_save_path, self.feature_matrix)
            
    def load_feature_matrix(self):
        self.feature_matrix = np.load(self.feature_matrix_save_path)
        
    def locate_feature_idx(self, featurizer_id, top_n=3, has_feature=True):
    
        feature_column = self.feature_matrix[:, featurizer_id]
        if not has_feature:
            feature_column = np.invert(feature_column)
        targets = np.where(feature_column)[0]

        num_examples = len(targets)
        if num_examples == 0:
            print(f"0 (not {top_n}) \t : {self.featurizers[featurizer_id].__name__}.")
            return []
        elif num_examples < top_n:
            print(f"{num_examples} (not {top_n}) \t : {self.featurizers[featurizer_id].__name__}.")
            top_n = num_examples

        # trying to find texts that don't have a lot of other 
        # competing features which may confound our results
        competing_features = self.feature_matrix[targets].sum(axis=1)
        sorted_idx = np.argsort(competing_features)

        return targets[sorted_idx][:top_n]
    
    def construct_feature_datasets(self, dataset, dataset_size=1000):
        feature_datasets = {}
        for i, featurizer in enumerate(graph_manager.featurizers):

            feature_present_idx = graph_manager.locate_feature_idx(i, dataset_size, has_feature=True)
            feature_absent_idx  = graph_manager.locate_feature_idx(i, len(feature_present_idx), has_feature=False)

            feature_present_dataset = dataset.select(feature_present_idx)
            feature_absent_dataset  = dataset.select(feature_absent_idx)

            feature_datasets[featurizer.__name__] = {
                "feature_present_idx"     : feature_present_idx,
                "feature_absent_idx"      : feature_absent_idx,
                "feature_present_dataset" : feature_present_dataset,
                "feature_absent_dataset"  : feature_absent_dataset,
            }
        return feature_datasets
            
def percent_dataset_changed(d1, d2):
    return sum([t1['text'] != t2['text'] for t1, t2 in zip(d1, d2)]) / len(d1)        

In [70]:
def compute_accuracy(predictions, labels):
    if not isinstance(labels, np.ndarray):
        labels = np.array(labels)
    if len(labels.shape) > 1:
        acc = acc_at_k(labels, predictions, k=2)       
    else:
        acc = accuracy_score(labels, np.argmax(predictions, -1))
    return acc

def vectorize(output):
    sorted_output = sorted(output, key=lambda d: d['label']) 
    probs = np.array([d['score'] for d in sorted_output])
    return probs

class EvalPipeline:
    
    def __init__(self):
        self.api = HfApi()
        self.pipe = None
        self.device = 0 if torch.cuda.is_available() else -1

    def find_model_for_dataset(self, dataset_name):
        
        model_filter = ModelFilter(
            task="text-classification",
            library="pytorch",
            # model_name=dataset_name,
            trained_dataset=dataset_name)

        model_id = next(iter(self.api.list_models(filter=model_filter)))

        if model_id:
            model_id = getattr(model_id, 'modelId')
            print('Using ' + model_id + ' to support evaluation.')
            self.pipe = pipeline("text-classification", 
                                 model=model_id, 
                                 device=self.device, 
                                 padding=True, 
                                 truncation=True,
                                 top_k=None)

    def extract_prediction_probabilities(self, dataset):
        output = self.pipe(dataset['text'])
        return np.stack([vectorize(o) for o in output])
    
    def extract_prediction_classes(self, dataset):
        return np.argmax(self.extract_prediction_probabilities(dataset), axis=1)
    
    def calculate_accuracy(self, dataset):
        preds = self.extract_prediction_probabilities(dataset)
        acc = compute_accuracy(preds, dataset["label"])
        return acc

## Load Data

In [5]:
dataset = load_dataset("glue", "sst2", split="train") # .select(range(100))
dataset = dataset.rename_column("sentence", "text")

Found cached dataset glue (C:/Users/Fabrice/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


## Extract AMRs + Featurize

In [6]:
graph_manager = AMRGraphManager()

# extract amr graphs
if graph_manager.graphs is None:
    graphs = graph_manager.text_to_amr(dataset["sentence"])
    graph_manager.save_graphs()

# compute boolean feature matrix
if graph_manager.feature_matrix is None:
    feature_matrix = graph_manager.generate_feature_matrix(graphs)
    graph_manager.save_feature_matrix()

## Construct datasets with / without features present

In [7]:
dataset_size = 1000

feature_datasets = {}
for i, featurizer in enumerate(graph_manager.featurizers):
    
    feature_present_idx = graph_manager.locate_feature_idx(i, dataset_size, has_feature=True)
    feature_absent_idx  = graph_manager.locate_feature_idx(i, len(feature_present_idx), has_feature=False)
    
    feature_present_dataset = dataset.select(feature_present_idx)
    feature_absent_dataset  = dataset.select(feature_absent_idx)
    
    feature_datasets[featurizer.__name__] = {
        "feature_present_idx"     : feature_present_idx,
        "feature_absent_idx"      : feature_absent_idx,
        "feature_present_dataset" : feature_present_dataset,
        "feature_absent_dataset"  : feature_absent_dataset,
    }
    

557 (not 1000) 	 : contains_exlamation.
78 (not 1000) 	 : contains_interrogative_clause.
709 (not 1000) 	 : contains_question.
622 (not 1000) 	 : contains_coreferences.
330 (not 1000) 	 : contains_accompanier.
200 (not 1000) 	 : contains_age.
532 (not 1000) 	 : contains_beneficiary.
742 (not 1000) 	 : contains_concession.
608 (not 1000) 	 : contains_condition.
888 (not 1000) 	 : contains_consist_of.
132 (not 1000) 	 : contains_destination.
577 (not 1000) 	 : contains_direction.
924 (not 1000) 	 : contains_duration.
251 (not 1000) 	 : contains_example.
92 (not 1000) 	 : contains_extent.
205 (not 1000) 	 : contains_instrument.
288 (not 1000) 	 : contains_medium.
650 (not 1000) 	 : contains_ord.
238 (not 1000) 	 : contains_path.
247 (not 1000) 	 : contains_polarity.
10 (not 1000) 	 : contains_polite.
7 (not 1000) 	 : contains_range.
1 (not 1000) 	 : contains_scale.
446 (not 1000) 	 : contains_source.
89 (not 1000) 	 : contains_subevent.
0 (not 1000) 	 : contains_wiki.


In [8]:
feature_datasets

{'contains_imperative': {'feature_present_idx': array([    0, 37052, 37166, 63623, 37169, 19311, 59715, 63680, 10356,
         10324, 37512, 37564, 52061, 10175, 51849, 37589, 63782, 37658,
          9933, 64067, 37737, 28958, 19471,  9748, 55875, 63725, 59905,
         10805, 36871, 36271, 11942, 29172, 11918, 29081, 11859, 63050,
         36426, 29070, 11631, 23682, 11552, 63208, 55863, 11316, 36727,
         11090, 63440, 11060, 11034, 11031, 52135, 11007, 36776, 10950,
          9708, 52179, 59438, 64190, 39316, 28143, 39468, 28130, 20241,
         59341, 27976, 39812, 20329, 54979, 64563, 56161,  8024, 51100,
         20511, 40234, 40412, 64685,  7724, 56389, 27566, 40555, 56475,
         40668, 40037, 28186, 51399, 56031, 38109, 19605,  9279, 28755,
          9196, 51645,  9167, 38430, 64234,  8971, 19703, 38545, 19718,
         51561, 51501, 51476, 28662, 28611, 64424, 28511,  8655, 51410,
          8538,  8492,  8489,  9644, 12002, 36246, 52196, 15752, 54142,
         32646, 17

## Transform Datasets

In [9]:
blacklist = [
    sibyl.transformations.text.emoji.emojify.Emojify,
    sibyl.transformations.text.emoji.emojify.AddPositiveEmoji,
    sibyl.transformations.text.emoji.emojify.AddNegativeEmoji,
    sibyl.transformations.text.emoji.demojify.Demojify,
    sibyl.transformations.text.emoji.demojify.RemovePositiveEmoji,
    sibyl.transformations.text.emoji.demojify.RemoveNegativeEmoji,
    sibyl.transformations.text.emoji.emojify.AddPositiveEmoji,
    sibyl.transformations.text.emoji.emojify.AddNegativeEmoji,
    sibyl.transformations.text.insertion.sentiment_phrase.InsertPositivePhrase,
    sibyl.transformations.text.insertion.sentiment_phrase.InsertNegativePhrase,
    sibyl.transformations.text.links.add_sentiment_link.AddPositiveLink,
    sibyl.transformations.text.links.add_sentiment_link.AddNegativeLink,
    sibyl.transformations.text.links.import_link_text.ImportLinkText,
    sibyl.transformations.text.negation.add_negation.AddNegation,
    sibyl.transformations.text.negation.remove_negation.RemoveNegation,
    sibyl.transformations.text.word_swap.change_synse.ChangeAntonym,
    sibyl.transformations.text.mixture.concept_mix.ConceptMix,
    sibyl.transformations.text.mixture.text_mix.TextMix,
    sibyl.transformations.text.mixture.text_mix.SentMix,
    sibyl.transformations.text.mixture.text_mix.WordMix,
    sibyl.transformations.text.generative.concept2sentence.Concept2Sentence
]
sibyl_transforms = [t for t in sibyl.TRANSFORMATIONS if t not in blacklist]

In [22]:
keep_originals = False
batch_size     = 10
task_name      = "sentiment"

for f_name, f_data in feature_datasets.items():

    transformed_datasets = {}
    for t in sibyl_transforms:
        
        aug = partial(augment_data, 
                      transform=Transform(t, task_name=task_name), 
                      keep_originals=keep_originals)
        
        feature_present_aug_dataset = f_data["feature_present_dataset"].map(aug, batched=True, batch_size=batch_size)
        feature_absent_aug_dataset  = f_data["feature_absent_dataset"].map(aug, batched=True, batch_size=batch_size)
        
        transformed_datasets[t.__name__] = {
            "feature_present_aug_dataset" : feature_present_aug_dataset,
            "feature_absent_aug_dataset"  : feature_absent_aug_dataset
        }
        
    feature_datasets[f_name]["transformed_datasets"] = transformed_datasets

Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-23e9558c4a33dddc.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-4506aec84247c6cb.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-17500acb143a56d3.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-35ddb982460573ac.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-d714243b45b8f28a.arrow


initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True


Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-10a175941082c2bf.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-031a8879e8aeaa6e.arrow


initializing class with task_name='sentiment', return_metadata=True


Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-37ebbd3f73ba4c96.arrow


...

  0%|          | 0/100 [00:00<?, ?ba/s]

initializing class with task_name='sentiment', return_metadata=True


  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', return_metadata=True
initializing class with task_name='sentiment', r

In [21]:
feature_datasets

{'contains_imperative': {'feature_present_idx': array([    0, 37052, 37166, 63623, 37169, 19311, 59715, 63680, 10356,
         10324, 37512, 37564, 52061, 10175, 51849, 37589, 63782, 37658,
          9933, 64067, 37737, 28958, 19471,  9748, 55875, 63725, 59905,
         10805, 36871, 36271, 11942, 29172, 11918, 29081, 11859, 63050,
         36426, 29070, 11631, 23682, 11552, 63208, 55863, 11316, 36727,
         11090, 63440, 11060, 11034, 11031, 52135, 11007, 36776, 10950,
          9708, 52179, 59438, 64190, 39316, 28143, 39468, 28130, 20241,
         59341, 27976, 39812, 20329, 54979, 64563, 56161,  8024, 51100,
         20511, 40234, 40412, 64685,  7724, 56389, 27566, 40555, 56475,
         40668, 40037, 28186, 51399, 56031, 38109, 19605,  9279, 28755,
          9196, 51645,  9167, 38430, 64234,  8971, 19703, 38545, 19718,
         51561, 51501, 51476, 28662, 28611, 64424, 28511,  8655, 51410,
          8538,  8492,  8489,  9644, 12002, 36246, 52196, 15752, 54142,
         32646, 17

In [23]:
feature_datasets

{'contains_imperative': {'feature_present_idx': array([    0, 37052, 37166, 63623, 37169, 19311, 59715, 63680, 10356,
         10324, 37512, 37564, 52061, 10175, 51849, 37589, 63782, 37658,
          9933, 64067, 37737, 28958, 19471,  9748, 55875, 63725, 59905,
         10805, 36871, 36271, 11942, 29172, 11918, 29081, 11859, 63050,
         36426, 29070, 11631, 23682, 11552, 63208, 55863, 11316, 36727,
         11090, 63440, 11060, 11034, 11031, 52135, 11007, 36776, 10950,
          9708, 52179, 59438, 64190, 39316, 28143, 39468, 28130, 20241,
         59341, 27976, 39812, 20329, 54979, 64563, 56161,  8024, 51100,
         20511, 40234, 40412, 64685,  7724, 56389, 27566, 40555, 56475,
         40668, 40037, 28186, 51399, 56031, 38109, 19605,  9279, 28755,
          9196, 51645,  9167, 38430, 64234,  8971, 19703, 38545, 19718,
         51561, 51501, 51476, 28662, 28611, 64424, 28511,  8655, 51410,
          8538,  8492,  8489,  9644, 12002, 36246, 52196, 15752, 54142,
         32646, 17

In [121]:
pipe = EvalPipeline()
pipe.find_model_for_dataset("sst2")

eval_results = []
for f_name, f_data in feature_datasets.items():
    
    if f_data["feature_present_dataset"].num_rows == 0:
        continue
    
    orig_T_acc = pipe.calculate_accuracy(f_data["feature_present_dataset"])
    orig_F_acc = pipe.calculate_accuracy(f_data["feature_absent_dataset"])
    
    orig_out = {
        "transform": "original",
        "featurizer": f_name,
        "num_samples": f_data["feature_present_dataset"].num_rows,
        "T_orig_acc": orig_T_acc,
        "F_orig_acc": orig_F_acc,
        "T_tran_acc": 0,
        "F_tran_acc": 0,
        "pct_T_changed": 0,
        "pct_F_changed": 0
    }
    
    eval_results.append(orig_out)
    
    for t_name, t_data in f_data["transformed_datasets"].items():
        print(f_name, t_name)
        
        tran_T_acc = pipe.calculate_accuracy(t_data["feature_present_aug_dataset"])
        tran_F_acc = pipe.calculate_accuracy(t_data["feature_absent_aug_dataset"])
        
        pct_T_changed = percent_dataset_changed(f_data["feature_present_dataset"], t_data["feature_present_aug_dataset"])
        pct_F_changed = percent_dataset_changed(f_data["feature_absent_dataset"],  t_data["feature_absent_aug_dataset"])
        
        tran_out = {
            "transform": t_name,
            "featurizer": f_name,
            "num_samples": f_data["feature_present_dataset"].num_rows,
            "T_orig_acc": orig_T_acc,
            "F_orig_acc": orig_F_acc,
            "T_tran_acc": tran_T_acc,
            "F_tran_acc": tran_F_acc,
            "pct_T_changed": pct_T_changed,
            "pct_F_changed": pct_F_changed
        }
        
        eval_results.append(tran_out)

Using distilbert-base-uncased-finetuned-sst-2-english to support evaluation.
contains_imperative ExpandContractions
contains_imperative ContractContractions
contains_imperative AddNeutralEmoji
contains_imperative RemoveNeutralEmoji
contains_imperative ChangeLocation


C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


contains_imperative ChangeName


C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


contains_imperative RandomInsertion


...

C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


contains_unit HomoglyphSwap


C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


contains_unit RandomSwap


C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\Fabrice\anaconda3\envs\fada\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [122]:
df = pd.DataFrame(eval_results)

In [124]:
df.to_csv("transform_feature_analysis.csv")